# 01_2. Formula 1 Qualifying Data Collection

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import time
start = time.time()

In [3]:
import pandas as pd
import numpy as np
from selenium import webdriver
import requests
import bs4
from bs4 import BeautifulSoup
import time

## Formula 1 Races from 1983 until 2021

In [4]:
qualifying_results = pd.DataFrame()
for year in list(range(1983,2021)):
    url = 'https://www.formula1.com/en/results.html/{}/races.html'
    r = requests.get(url.format(year))
    soup = BeautifulSoup(r.text, 'html.parser')
    
    year_links = []
    for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
        link = page.get('href')
        if f'/en/results.html/{year}/races/' in link: 
            year_links.append(link)

    year_df = pd.DataFrame()
    new_url = 'https://www.formula1.com{}'
    for n, link in list(enumerate(year_links)):
        link = link.replace('race-result.html', 'starting-grid.html')
        df = pd.read_html(new_url.format(link))
        df = df[0]
        df['season'] = year
        df['round'] = n+1
        for col in df:
            if 'Unnamed' in col:
                df.drop(col, axis = 1, inplace = True)

        year_df = pd.concat([year_df, df])

    qualifying_results = pd.concat([qualifying_results, year_df])
    
print(qualifying_results.shape)

(14898, 7)


In [5]:
len(qualifying_results)

14898

In [6]:
qualifying_results.head()

Pos  No                Driver             Car      Time  season  round
0    1   1    Keke  Rosberg  ROS  Williams Honda  1:34.526    1983      1
1    2  15     Alain  Prost  PRO         Renault  1:34.672    1983      1
2    3  27  Patrick  Tambay  TAM         Ferrari  1:34.758    1983      1
3    4   5   Nelson  Piquet  PIQ     Brabham BMW  1:35.114    1983      1
4    5  35   Derek  Warwick  WAR    Toleman Hart  1:35.206    1983      1

In [7]:
qualifying_results.tail()

Pos  No                   Driver                        Car      Time  \
15   16  63     George  Russell  RUS          Williams Mercedes  1:38.045   
16   17  51  Pietro  Fittipaldi  FIT               Haas Ferrari  1:38.173   
17   18   6    Nicholas  Latifi  LAT          Williams Mercedes  1:38.443   
18   19  11       Sergio  Perez  PER  Racing Point BWT Mercedes       NaN   
19   20  20    Kevin  Magnussen  MAG               Haas Ferrari  1:37.863   

    season  round  
15    2020     17  
16    2020     17  
17    2020     17  
18    2020     17  
19    2020     17

In [8]:
qualifying_results.rename(columns = {'Pos': 'grid_position', 'Driver': 'driver_name', 'Car': 'car',
                                     'Time': 'qualifying_time'}, inplace = True)

In [9]:
qualifying_results.drop('No', axis = 1, inplace = True)

In [10]:
qualifying_results.to_csv('./data/qualifying.csv', index = False)

In [11]:
end = time.time()

In [12]:
import datetime
str(datetime.timedelta(seconds=(end - start)))

'0:02:38.707951'

In [13]:
print(str(end - start)+" seconds")

158.70795059204102 seconds
